## Sentinel-2 Multi-Spectral Imager (MSI) Demo

In this tutorial, we will look at MSI data using SatPy. More information about the instrument can be found here: https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-2-msi/msi-instrument

In [1]:
# Ignore warning and debug messages
# NB: If things go wrong, you can turn the debugger on and turn the warnings filter off.
import warnings
warnings.catch_warnings()
warnings.simplefilter("ignore")
from satpy.utils import debug_off; debug_off()

As before, we can read in the appropriate files using SatPy's utility function:

In [2]:
from satpy import find_files_and_readers

path_msi = './data/msi/'
files = find_files_and_readers(reader='msi_safe',
                               base_dir=path_msi)

Sentinel-2 MSI files come in the `.SAFE` file format, which is actually series of directories containing geolocated files (.jp2) and metadata (.xml). To view the filenames we can print the files as before:

In [3]:
# Check the list of files
print(files)

{'msi_safe': ['./data/msi/S2B_MSIL1C_20230417T230139_N0509_R015_T57FVV_20230417T234656.SAFE/GRANULE/L1C_T57FVV_A031930_20230417T230134/MTD_TL.xml', './data/msi/S2B_MSIL1C_20230417T230139_N0509_R015_T57FVV_20230417T234656.SAFE/MTD_MSIL1C.xml', './data/msi/S2B_MSIL1C_20230417T230139_N0509_R015_T57FVV_20230417T234656.SAFE/GRANULE/L1C_T57FVV_A031930_20230417T230134/IMG_DATA/T57FVV_20230417T230139_B04.jp2', './data/msi/S2B_MSIL1C_20230417T230139_N0509_R015_T57FVV_20230417T234656.SAFE/GRANULE/L1C_T57FVV_A031930_20230417T230134/IMG_DATA/T57FVV_20230417T230139_B06.jp2', './data/msi/S2B_MSIL1C_20230417T230139_N0509_R015_T57FVV_20230417T234656.SAFE/GRANULE/L1C_T57FVV_A031930_20230417T230134/IMG_DATA/T57FVV_20230417T230139_B8A.jp2', './data/msi/S2B_MSIL1C_20230417T230139_N0509_R015_T57FVV_20230417T234656.SAFE/GRANULE/L1C_T57FVV_A031930_20230417T230134/IMG_DATA/T57FVV_20230417T230139_B07.jp2', './data/msi/S2B_MSIL1C_20230417T230139_N0509_R015_T57FVV_20230417T234656.SAFE/GRANULE/L1C_T57FVV_A031930_

This looks like a bit of a mess but with SatPy's `Scene` object, we can make sense of what we have:

In [4]:
from satpy import Scene

scn = Scene(files)
scn.available_dataset_names()

['B01',
 'B02',
 'B03',
 'B04',
 'B05',
 'B06',
 'B07',
 'B08',
 'B09',
 'B10',
 'B11',
 'B12',
 'B8A',
 'satellite_azimuth_angle',
 'satellite_zenith_angle',
 'solar_azimuth_angle',
 'solar_zenith_angle']

Here we can see the 13 spectral channels that MSI has, along with some infromation regarding the sensor geometries. Let's get some more information about a particular band:

In [5]:
scn.load(['B8A'])
print(scn)

<xarray.DataArray (y: 5490, x: 5490)>
dask.array<mul, shape=(5490, 5490), dtype=float64, chunksize=(4096, 4096), chunktype=numpy.ndarray>
Coordinates:
    band         int64 1
  * x            (x) float64 4e+05 4e+05 4e+05 ... 5.097e+05 5.097e+05 5.098e+05
  * y            (y) float64 4e+06 4e+06 4e+06 ... 3.89e+06 3.89e+06 3.89e+06
    spatial_ref  int64 0
    crs          object PROJCRS["WGS 84 / UTM zone 57S",BASEGEOGCRS["WGS 84",...
Attributes: (12/16)
    name:                 B8A
    sensor:               MSI
    wavelength:           0.865 µm (0.855-0.875 µm)
    resolution:           20
    calibration:          reflectance
    file_type:            safe_granule
    ...                   ...
    start_time:           2023-04-17 23:01:39
    end_time:             2023-04-17 23:01:39
    reader:               msi_safe
    area:                 Area ID: 57FVV\nDescription: On-the-fly area\nProje...
    _satpy_id:            DataID(name='B8A', wavelength=WavelengthRange(min=0...
  

We can also look at the available composite names as before:

In [6]:
scn.available_composite_names()

['cloud_phase',
 'cloud_phase_raw',
 'day_essl_colorized_low_level_moisture',
 'day_essl_low_level_moisture',
 'essl_colorized_low_level_moisture',
 'essl_low_level_moisture',
 'natural_color',
 'true_color',
 'true_color_desert',
 'true_color_land',
 'true_color_marine_clean',
 'true_color_marine_tropical',
 'true_color_raw']

Let's see what a true colour image looks like at the native MSI resolution:

In [ ]:
from dask.diagnostics import ProgressBar

scn.load(['true_color'])
lcn = scn.resample(scn.finest_area())

with ProgressBar():
    lcn.save_dataset('true_color', tiled=True, filename='./data/geotiff/msi_tc.tif')

Now, let's resample to the same area that we defined before for MODIS and AHI, so that we can get some appreciation for the differences in spatial resolution among the three sensors:

In [8]:
from dask.diagnostics import ProgressBar
from pyresample import create_area_def

# extent = [lonmin, lonmax, latmin, latmax]
extent = [158.6, 159.2, -54.8, -54.4]
area_def = create_area_def('Macquarie Island', "+proj=eqc", units="degrees",
                           area_extent=(extent[0], extent[2], extent[1], extent[3]), resolution=0.0001)
scn.load(['true_color'])
lcn = scn.resample(area_def)

with ProgressBar():
    lcn.save_dataset('true_color', tiled=True, filename='./data/geotiff/msi_mac_island_tc.tif')

[########################################] | 100% Completed | 23.32 s


As we can see, the MSI data has very high spatial resolution and could be used to study vegetation changes on the island. One comment metric used to assess vegetation health is the Normalised Difference Vegetation Index. To compute the NDVI, we do:

In [ ]:
scn.load(['B04', 'B08'])
lcn = scn.resample(area_def)
# Get the RED and NIR data arrays
red = lcn['B04'].values
nir = lcn['B08'].values
# Compute NDVI
ndvi = (nir-red) / (nir+red)

Finally, we can plot the NDVI using standard matplotlib methods:

In [ ]:
# Plot the data
import matplotlib.pyplot as plt

plt.imshow(ndvi, cmap='YlGn', vmin=0, vmax=1)
plt.show()

## Exercise
Can you compute and plot a different spectral index over Macquarie Island? Some examples are here: https://giscrack.com/list-of-spectral-indices-for-sentinel-and-landsat/